<a href="https://colab.research.google.com/github/ArturSViana/Copa-do-Mundo-2022/blob/main/PrevisaoCopa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [198]:
import pandas as pd
import numpy as np
import random
from scipy.stats import poisson

In [199]:
selecoes = pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name = 'selecoes').set_index('Seleção')
jogos = pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name = 'jogos')
selecoes.head()

,Grupo,GrupoNumero,NomeEmIngles,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
Seleção,,,,,,,,,,,,
Catar,A,A1,Qatar,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Equador,A,A2,Ecuador,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,A,A3,Senegal,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Holanda,A,A4,Netherlands,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Inglaterra,B,B1,England,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [200]:
jogos.head()

,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR


In [201]:
fifa = selecoes['PontosRankingFIFA']
fifa

Seleção
Catar             1442
Equador           1464
Senegal           1585
Holanda           1679
Inglaterra        1737
Irã               1559
Estados Unidos    1635
País de Gales     1582
Argentina         1771
Arábia Saudita    1436
México            1650
Polônia           1546
França            1765
Dinamarca         1665
Tunísia           1508
Austrália         1484
Espanha           1717
Alemanha          1659
Japão             1555
Costa Rica        1500
Bélgica           1822
Canadá            1474
Marrocos          1558
Croácia           1632
Brasil            1838
Sérvia            1550
Suíça             1621
Camarões          1485
Portugal          1679
Gana              1393
Uruguai           1641
Coreia do Sul     1526
Name: PontosRankingFIFA, dtype: int64

In [202]:
# eq de reta: y - y0 = m*(x-x0)

melhor_selecao, pior_selecao = max(fifa), min(fifa)
ponto_melhor, ponto_pior = 1, 0.15
b1 = (ponto_melhor - ponto_pior) / (melhor_selecao - pior_selecao)
print(b1)
b0 = ponto_melhor - melhor_selecao*b1
print(b0)
forca = b1*fifa + b0
forca

0.0019101123595505617
-2.5107865168539325


Seleção
Catar             0.243596
Equador           0.285618
Senegal           0.516742
Holanda           0.696292
Inglaterra        0.807079
Irã               0.467079
Estados Unidos    0.612247
País de Gales     0.511011
Argentina         0.872022
Arábia Saudita    0.232135
México            0.640899
Polônia           0.442247
França            0.860562
Dinamarca         0.669551
Tunísia           0.369663
Austrália         0.323820
Espanha           0.768876
Alemanha          0.658090
Japão             0.459438
Costa Rica        0.354382
Bélgica           0.969438
Canadá            0.304719
Marrocos          0.465169
Croácia           0.606517
Brasil            1.000000
Sérvia            0.449888
Suíça             0.585506
Camarões          0.325730
Portugal          0.696292
Gana              0.150000
Uruguai           0.623708
Coreia do Sul     0.404045
Name: PontosRankingFIFA, dtype: float64

In [217]:
def Pontos(gols1, gols2):
  if gols1 > gols2:
    pontos1, pontos2 = 3, 0
  if gols1 == gols2:
    pontos1, pontos2 = 1, 1
  if gols1 < gols2:
    pontos1, pontos2 = 0, 3
  return pontos1, pontos2


def MediasPoisson(selecao1, selecao2):
  '''
    essa função calcula a quanto cada equipe tem de participação na média geral de gols na copa,
    esta média geral é arbitrária mas tem embasamento em dados históricos das médias de gols nas edições de copas do mundo
  '''
  forca1 = forca[selecao1]
  forca2 = forca[selecao2]
  m_gols_jogo = 2.75
  media1 = forca1 * m_gols_jogo / (forca1+forca2)
  media2 = forca2 * m_gols_jogo / (forca1+forca2)
  return media1, media2


def DistribuicaoPlacares(media):
  '''
    probabilidade de um time x fazer 0, 1, 2, 3 gols, ...
  '''
  probs = []
  for i in range(8):
    probs.append(poisson.pmf(i, media))

  probs.append(1-sum(probs))
  probs_df = pd.Series(probs, index = ['0', '1', '2', '3', '4', '5', '6', '7', '8+'])
  return probs_df


def Jogo(selecao1, selecao2):
  media1, media2 = MediasPoisson(selecao1, selecao2)
  gols1 = int(np.random.poisson(lam = media1, size = 1)) #gera um número aleatório (gols) dada a mediaPoisson do time, size=1 significa 1 geração
  gols2 = int(np.random.poisson(lam = media2, size = 1)) 
  placar = f'{gols1} X {gols2}'
  pontos1, pontos2 = Pontos(gols1, gols2)
  saldo1 = gols1 - gols2
  saldo2 = gols2 - gols1
  if gols1 > gols2:
    res = selecao1
  if gols1 < gols2:
    res = selecao2
  if gols1 == gols2:
    res = 'empate'
  return {selecao1: [gols1, pontos1, saldo1], selecao2: [gols2, pontos2, saldo2], 'Placar': placar,
          'Resultado': res}


def ProbabilidadesPartida(selecao1, selecao2):
  media1, media2 = MediasPoisson(selecao1, selecao2)
  prob_gols1, prob_gols2 = DistribuicaoPlacares(media1), DistribuicaoPlacares(media2)
  matriz = np.outer(prob_gols1, prob_gols2) #monta a matriz de probabilidade entre os dois times
  nomes = ['0', '1', '2', '3', '4', '5', '6', '7', '8+']

  #este df foi feito para visualizar qual time está em cada eixo, isso nos ajudará a com a probabilidade de vitória dos times
  matriz_df = pd.DataFrame(matriz, columns = nomes, index = nomes)
  matriz_df.index = pd.MultiIndex.from_product([[selecao1], matriz_df.index])
  matriz_df.columns = pd.MultiIndex.from_product([[selecao2], matriz_df.columns]) 

  empate = np.trace(matriz).sum() #pega a diagonal da matriz (número de gols iguais entre as equipes -> empate) e soma as probabilidades
  vitoria1 = np.tril(matriz).sum() - empate #pega a parte inferior da matriz e tira a diagonal, depois soma as probabilidades
  vitoria2 = np.triu(matriz).sum() - empate #pega a parte superior da matriz e tira a diagonal, depois soma as probabilidades
  probs = np.around([vitoria1, empate , vitoria2], 3)
  probsp = [f'{100*i:.1f}%' for i in probs]

  output = {
      'Seleção 1': selecao1,
      f'Força - {selecao1}': forca[selecao1],
      f'Média de Poisson - {selecao1}': media1,
      'Seleção 2': selecao2,
      f'Força - {selecao2}': forca[selecao2],
      f'Média de Poisson - {selecao2}': media2,
      'Probabilidades (V1, E, V2)': probsp,
      'Matriz de Probabilidades': matriz_df
      }

  return matriz_df

In [204]:
media1, media2 = MediasPoisson('Brasil', 'Catar')
media1

2.211329960245754

In [218]:
ProbabilidadesPartida('Brasil', 'Argentina')

Argentina                                                    \
                  0         1         2         3         4         5   
Brasil 0   0.063928  0.081892  0.052452  0.022397  0.007173  0.001838   
       1   0.093910  0.120299  0.077051  0.032901  0.010537  0.002699   
       2   0.068977  0.088359  0.056594  0.024166  0.007739  0.001983   
       3   0.033776  0.043267  0.027712  0.011833  0.003790  0.000971   
       4   0.012404  0.015890  0.010177  0.004346  0.001392  0.000357   
       5   0.003644  0.004668  0.002990  0.001277  0.000409  0.000105   
       6   0.000892  0.001143  0.000732  0.000313  0.000100  0.000026   
       7   0.000187  0.000240  0.000154  0.000066  0.000021  0.000005   
       8+  0.000041  0.000052  0.000034  0.000014  0.000005  0.000001   

                                                     
                      6             7            8+  
Brasil 0   3.923318e-04  7.179676e-05  1.336970e-05  
       1   5.763353e-04  1.054694e-04  1.964009e-05  
       2   4.233181e-04  7.746724e-05  1.442564e-05  
       3   2.072847e-04  3.793311e-05  7.063752e-06  
       4   7.612527e-05  1.393093e-05  2.594162e-06  
       5   2.236560e-05  4.092906e-06  7.621645e-07  
       6   5.475842e-06  1.002079e-06  1.866032e-07  
       7   1.149144e-06  2.102934e-07  3.916000e-08  
       8+  2.512816e-07  4.598455e-08  8.563057e-09

# Simular as fases da Copa

### Fase de Grupos

In [220]:
def SimulacaoGrupos(grupo):
  times = list(selecoes[selecoes['Grupo']== grupo].index)
  time1, time2, time3, time4 = times
  jogo1 = Jogo(time1, time2)
  jogo2 = Jogo(time3, time4)
  jogo3 = Jogo(time1, time3)
  jogo4 = Jogo(time2, time4)
  jogo5 = Jogo(time1, time4)
  jogo6 = Jogo(time2, time3)

  pt1, pt2, pt3, pt4 = 0, 0, 0, 0
  gp1, gp2, gp3, gp4 = 0, 0, 0, 0
  s1, s2, s3, s4 = 0, 0, 0, 0

  gp1, pt1, s1, gp2, pt2, s2 = gp1 + jogo1[time1][0], pt1 + jogo1[time1][1], s1 + jogo1[time1][2], gp2 + jogo1[time2][0], pt2 + jogo1[time2][1], s2 + jogo1[time2][2]
  gp3, pt3, s3, gp4, pt4, s4 = gp3 + jogo2[time3][0], pt3 + jogo2[time3][1], s3 + jogo2[time3][2], gp4 + jogo2[time4][0], pt4 + jogo2[time4][1], s4 + jogo2[time4][2]
  gp1, pt1, s1, gp3, pt3, s3 = gp1 + jogo3[time1][0], pt1 + jogo3[time1][1], s1 + jogo3[time1][2], gp3 + jogo3[time3][0], pt3 + jogo3[time3][1], s3 + jogo3[time3][2]
  gp2, pt2, s2, gp4, pt4, s4 = gp2 + jogo4[time2][0], pt2 + jogo4[time2][1], s2 + jogo4[time2][2], gp4 + jogo4[time4][0], pt4 + jogo4[time4][1], s4 + jogo4[time4][2]
  gp1, pt1, s1, gp4, pt4, s4 = gp1 + jogo5[time1][0], pt1 + jogo5[time1][1], s1 + jogo5[time1][2], gp4 + jogo5[time4][0], pt4 + jogo5[time4][1], s4 + jogo5[time4][2]
  gp2, pt2, s2, gp3, pt3, s3 = gp2 + jogo6[time2][0], pt2 + jogo6[time2][1], s2 + jogo6[time2][2], gp3 + jogo6[time3][0], pt3 + jogo6[time3][1], s3 + jogo6[time3][2]
  
  tabela = pd.DataFrame([[pt1, s1, gp1], [pt2, s2, gp2], [pt3, s3, gp3], [pt4, s4, gp4]], index = times, columns = ['Pontos', 'Saldo de Gols', 'Gols Pró'])
  tabela = tabela.sort_values(by=['Pontos', 'Saldo de Gols', 'Gols Pró'], ascending=False)
  tabela['Posição'] = [1, 2, 3, 4]

  confrontos = [f'{time1} x {time2}', f'{time3} x {time4}', f'{time1} x {time3}', f'{time2} x {time4}', f'{time1} x {time4}', f'{time2} x {time3}']
  resultados = [jogo1['Placar'], jogo2['Placar'], jogo3['Placar'], jogo4['Placar'], jogo5['Placar'], jogo6['Placar']]

  partidas = pd.DataFrame([resultados], index = ['Resultados'], columns=confrontos).T
  return tabela, partidas

In [221]:
grupo = SimulacaoGrupos('G')
display(grupo[0])
display(grupo[1])

,Pontos,Saldo de Gols,Gols Pró,Posição
Brasil,9,6,6,1
Camarões,4,0,4,2
Sérvia,3,-1,3,3
Suíça,1,-5,2,4


,Resultados
Brasil x Sérvia,2 X 0
Suíça x Camarões,2 X 2
Brasil x Suíça,3 X 0
Sérvia x Camarões,1 X 2
Brasil x Camarões,1 X 0
Sérvia x Suíça,2 X 0


### Segunda Fase

Criação do armazenamento das informações 

In [223]:
cols = ['1st', '2nd', '3th', '4th', 'Oitavas', 'Quartas', 'Semis', 'Final', 'Campeão']
n = selecoes.shape[0]
m = len(cols)
aux = np.array(np.zeros(n*m).reshape(n,m))
info = pd.DataFrame(aux, index = selecoes.index, columns = cols).astype(int)


In [224]:
def SimulacaoMataMata(selecao1, selecao2):
  res = Jogo(selecao1, selecao2)['Resultado']
  if res == 'empate':
    res = random.sample([selecao1, selecao2], 1)[0]
    return res
  else:
    return res

In [225]:
SimulacaoMataMata('Brasil', 'França')

'França'

In [258]:
def SimulaCopa(selecoes):
  '''
    simula uma única copa
  '''
  cols = ['1st', '2nd', '3th', '4th', 'Oitavas', 'Quartas', 'Semis', 'Final', 'Campeão']
  n = selecoes.shape[0]
  m = len(cols)
  aux = np.array(np.zeros(n*m).reshape(n,m))
  info = pd.DataFrame(aux, index = selecoes.index, columns = cols).astype(int)
  top16=[]
  for grupo in list('ABCDEFGH'):
  # realização das simulações da fase de grupos
    a = SimulacaoGrupos(grupo)[0]
    res = a.index.tolist()
    top16 += a.index[:2].tolist()

  # input dos resultados da fase de grupo e que avançaram para as oitavas
    info['1st'][res[0]] = 1
    info['2nd'][res[1]] = 1
    info['3th'][res[2]] = 1
    info['4th'][res[3]] = 1

    info['Oitavas'][info.loc[:, '1st'] == 1] = 1
    info['Oitavas'][info.loc[:, '2nd'] == 1] = 1

  # input dos dos resultados das oitavas e que avançaram para as quartas
  of1 = SimulacaoMataMata(top16[0], top16[3]) #1A x 2B
  of2 = SimulacaoMataMata(top16[4], top16[7]) #1C x 2D
  of3 = SimulacaoMataMata(top16[8], top16[11]) #1E x 2F
  of4 = SimulacaoMataMata(top16[12], top16[15]) #1G x 2H
  of5 = SimulacaoMataMata(top16[2], top16[1]) #1B x 2A
  of6 = SimulacaoMataMata(top16[6], top16[5]) #1D x 2C
  of7 = SimulacaoMataMata(top16[10], top16[9]) #1F x 2E
  of8 = SimulacaoMataMata(top16[14], top16[13]) #1H x 2G

  classificados_das_oitavas = [of1, of2, of3, of4, of5, of6, of7, of8]
  info['Quartas'][classificados_das_oitavas]=1

  # input dos dos resultados das quartas e que avançaram para as semis
  qf1 = SimulacaoMataMata(of1, of2)
  qf2 = SimulacaoMataMata(of3, of4)
  qf3 = SimulacaoMataMata(of5, of6)
  qf4 = SimulacaoMataMata(of7, of8)

  classificados_das_quartas = [qf1, qf2, qf3, qf4]
  info['Semis'][classificados_das_quartas]=1

  # input dos dos resultados das semis e que avançaram para a final
  sf1 = SimulacaoMataMata(qf1, qf2)
  sf2 = SimulacaoMataMata(qf3, qf4)

  classificados_das_semis = [sf1, sf2]
  info['Final'][classificados_das_semis]=1

  # input dos dos resultados da final e que foi campeão
  final = SimulacaoMataMata(sf1, sf2)

  campeao = final
  info['Campeão'][campeao]=1


  return info

In [230]:
def SimulacaoMassa(dados, S):
  print('Iniciando as simulações:')
  info = SimulaCopa(selecoes)
  for i in range(S-1):
    info += SimulaCopa(selecoes)
    if (i+2)%(S/10)==0:
      print(f'{100*(i+2)/S}% das simulações realizadas')
  print(f'{S} simulações realizadas!')
  return info.sort_values(by='Campeão', ascending = False)/S

In [259]:
sim = SimulacaoMassa(selecoes, 10000)

Iniciando as simulações:
10000 simulações realizadas!


### Insights Extras

- Probabilidade das seleções caírem nas etapas da Copa

In [261]:
parada = pd.DataFrame()
parada['Cair 1 fase'] = 1 - sim['Oitavas']
parada['Cair nas oitavas'] = sim['Oitavas'] - sim['Quartas']
parada['Cair nas quartas'] = sim['Quartas'] - sim['Semis']
parada['Cair nas semis'] = sim['Semis'] - sim['Final']
parada['Cair na final'] = sim['Final'] - sim['Campeão']
parada

,Cair 1 fase,Cair nas oitavas,Cair nas quartas,Cair nas semis,Cair na final
Seleção,,,,,
Brasil,0.1553,0.2825,0.2117,0.1303,0.0842
Bélgica,0.1646,0.3001,0.2084,0.1281,0.0788
Argentina,0.1805,0.3246,0.1938,0.1327,0.0747
França,0.1884,0.3313,0.1840,0.1282,0.0763
Inglaterra,0.2901,0.2587,0.2033,0.1124,0.0634
Espanha,0.2629,0.3266,0.1929,0.1003,0.0567
Holanda,0.1758,0.3833,0.2160,0.1178,0.0539
Portugal,0.2201,0.3804,0.2011,0.0960,0.0528
Dinamarca,0.3083,0.3423,0.1654,0.1000,0.0451


- Probabilidade das seleções avançarem nas etapas da Copa

In [262]:
avanco=pd.DataFrame()
avanco['Avançar na 1 fase'] = sim['Oitavas']
avanco['Avançar nas oitavas'] = sim['Quartas'] / sim['Oitavas']
avanco['Avançar nas quartas'] = sim['Semis'] / sim['Quartas']
avanco['Avançar nas semis'] = sim['Final'] / sim['Semis']
avanco['Avançar na final'] = sim['Campeão'] / sim['Final']
avanco

,Avançar na 1 fase,Avançar nas oitavas,Avançar nas quartas,Avançar nas semis,Avançar na final
Seleção,,,,,
Brasil,0.8447,0.665562,0.623444,0.628245,0.617620
Bélgica,0.8354,0.640771,0.610686,0.608137,0.603622
Argentina,0.8195,0.603905,0.608406,0.559283,0.556413
França,0.8116,0.591794,0.616906,0.567330,0.546104
Inglaterra,0.7099,0.635582,0.549424,0.546591,0.532103
Espanha,0.7371,0.556912,0.530085,0.539062,0.516624
Holanda,0.8242,0.534943,0.510093,0.476212,0.496732
Portugal,0.7799,0.512245,0.496621,0.516129,0.484375
Dinamarca,0.6917,0.505132,0.526617,0.456522,0.463095
